In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

warnings.simplefilter(action="ignore", category=FutureWarning)
sns.set_theme(color_codes=True, style="whitegrid")

In [2]:
data = pd.read_csv("characteristics_data_feb2017.csv")
data["date"] = pd.to_datetime(data["date"])

In [3]:
data.sort_values(by=["date"], ascending=[True], inplace=True)
data.head()

,Unnamed: 0,yy,mm,date,permno,ret,q10,q20,q50,prc,...,beta,cum_return_12_2,cum_return_12_7,cum_return_1_0,cum_return_36_13,idio_vol,spread_mean,suv,rel_to_high_price,lev
214585,214586,1962,7,1962-07-31,19940,-0.010899,10760.7,18723.6,59289.5,45.375,...,0.599815,-0.029524,0.144549,-0.028952,0.498805,0.024990,0.020291,0.556709,0.790948,0.013437
286044,286045,1962,7,1962-07-31,25160,-0.039216,10760.7,18723.6,59289.5,36.750,...,0.835357,-0.204031,-0.000321,-0.125714,0.266304,0.016088,0.025290,-0.289882,0.608350,0.328550
290255,290256,1962,7,1962-07-31,25478,-0.056452,10760.7,18723.6,59289.5,14.625,...,-0.012614,0.258911,0.234755,0.066114,0.155366,0.035262,0.045948,0.111966,0.885714,0.000000
214586,214587,1962,8,1962-08-31,19940,0.104683,11999.4,20397.0,64764.0,50.125,...,0.581311,-0.073672,0.065842,-0.010899,0.375561,0.010962,0.008705,2.012783,0.782328,0.013437
286045,286046,1962,8,1962-08-31,25160,0.027211,11999.4,20397.0,64764.0,37.500,...,0.794090,-0.288208,0.031714,-0.039216,0.121589,0.015801,0.014179,1.031443,0.584493,0.328550


In [4]:
def valid_entries(period, data):
    """Return ids of entries that exist throughout test and validation period"""

    if max(period) > data["date"].max():
        raise Exception(f"No data beyond 2014-05-31 available")

    elif min(period) < data["date"].min():
        raise Exception(f"No data before 1962-07-31 available")

    filtered_data = data.loc[data["date"].isin(period)]
    unique_date_count = filtered_data["date"].nunique()

    valid_ids = (
        filtered_data.groupby("permno")
        .filter(lambda x: x["date"].nunique() == unique_date_count)["permno"]
        .unique()
        .tolist()
    )

    return valid_ids

In [10]:
def get_top_N_stocks(start_year, end_year, data=data, N=500):
    """Get top N stocks by market cap in a given time period, only for stocks that have existed in the entire period (test data) and the year after given period (validation)"""

    period = pd.date_range(
        start=f"{start_year}-01-01", end=f"{end_year+1}-12-31", freq="M"
    )

    valid_ids = valid_entries(period, data)

    test_data = data.loc[
        data["date"].dt.year.between(start_year, end_year)
        & data["permno"].isin(valid_ids)
    ]
    validation_data = data.loc[
        data["date"].dt.year.eq(end_year + 1) & data["permno"].isin(valid_ids)
    ]

    top_N_test = (
        test_data.groupby("date")
        .apply(lambda x: x.nlargest(N, "lme"), include_groups=False)
        .reset_index(drop=True)
    )
    top_N_validation = (
        validation_data.groupby("date")
        .apply(lambda x: x.nlargest(N, "lme"), include_groups=False)
        .reset_index(drop=True)
    )

    return top_N_test, top_N_validation

,Unnamed: 0,yy,mm,permno,ret,q10,q20,q50,prc,a2me,...,beta,cum_return_12_2,cum_return_12_7,cum_return_1_0,cum_return_36_13,idio_vol,spread_mean,suv,rel_to_high_price,lev
0,98460,2012,1,11850,-0.012034,236967.261576,493150.660030,2.014954e+06,83.74000,0.820451,...,0.760573,0.125594,0.124990,0.053705,-0.036167,0.004725,0.000122,-2.156150,0.963401,0.092763
1,144022,2012,1,14593,0.127111,236967.261576,493150.660030,2.014954e+06,456.48001,0.253066,...,0.685369,0.184896,0.040644,0.059655,2.779262,0.006368,0.000280,-1.262987,0.959170,0.000000
2,7074,2012,1,10107,0.137519,236967.261576,493150.660030,2.014954e+06,29.53000,0.367176,...,0.644267,-0.059187,-0.056831,0.014855,0.502204,0.007695,0.000333,-1.619439,0.899203,0.113962
3,121938,2012,1,12490,0.047422,236967.261576,493150.660030,2.014954e+06,192.60001,0.622238,...,0.687130,0.302669,0.178819,-0.021915,0.813016,0.009401,0.000098,-1.528268,0.945107,0.553977
4,143373,2012,1,14541,-0.030827,236967.261576,493150.660030,2.014954e+06,103.12000,1.006180,...,0.896605,0.161146,0.143272,0.034818,0.331403,0.008646,0.000151,-1.520525,0.970272,0.098458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,537704,2012,12,43123,0.166539,255899.741937,532388.099824,2.077467e+06,30.36000,1.189396,...,2.328414,-0.465064,-0.351585,-0.006452,0.517576,0.020586,0.000443,0.474307,0.507168,0.378783
5996,909086,2012,12,75070,0.057959,255899.741937,532388.099824,2.077467e+06,41.17000,1.237952,...,1.386378,-0.187458,-0.123680,0.122478,0.628205,0.022808,0.000383,1.032533,0.795873,0.177240
5997,1089514,2012,12,79879,0.112943,255899.741937,532388.099824,2.077467e+06,13.50000,0.812386,...,2.134200,-0.117031,-0.075592,0.251160,0.485792,0.021997,0.000899,0.114999,0.812458,0.211516
5998,1100889,2012,12,80167,-0.100382,255899.741937,532388.099824,2.077467e+06,54.22000,0.820770,...,1.307306,0.547938,0.486074,0.042734,1.153403,0.017181,0.000325,-0.385659,0.824036,0.209844
